In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


## **Loading Dataset**

In [ ]:
!pip install antropy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/left_hip_DatasetWithlabels.csv')#,index_col='time_s',parse_dates=['time_s'])
df = df.drop(['Unnamed: 0'], 1)
# df = df.drop(['loc_id'], 1)
#Converting to category type 

df['subj_id'] = df['subj_id'].astype('category')
df['subj_id'] = df['subj_id'].cat.codes

df['label'] = df['label'].astype('category')
df['label'] = df['label'].cat.codes

df['loc_id'] = df['loc_id'].astype('category')
df['loc_id'] = df['loc_id'].cat.codes

In [ ]:
df

,subj_id,loc_id,time_s,x,y,z,magnitude,label
0,15,0,0.00,-0.836,0.305,0.402,0.976486,0
1,15,0,0.01,-0.859,0.172,0.316,0.931301,0
2,15,0,0.02,-0.781,0.145,0.242,0.830391,0
3,15,0,0.03,-0.605,0.191,0.207,0.667349,0
4,15,0,0.04,-0.441,0.250,0.207,0.547567,0
...,...,...,...,...,...,...,...,...
647607,31,0,206.71,0.098,1.059,0.090,1.067326,0
647608,31,0,206.72,0.105,1.070,0.074,1.077683,0
647609,31,0,206.73,0.125,1.094,0.090,1.104790,0
647610,31,0,206.74,0.141,1.129,0.090,1.141325,0


## **TEST AND TRAIN DATASET**

In [ ]:
X_train = df[df.subj_id.isin( df.subj_id.unique()[:10] ) ] 
y_train = X_train['label'] 
X_train = X_train.drop(['label'], 1)

In [ ]:
X_test = df[df.subj_id.isin( df.subj_id.unique()[5:] )]
y_test = X_test['label'] 
X_test = X_test.drop(['label'], 1)

## **RANDOM FOREST**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel

***STANDARD SCALING***

In [ ]:
#SCALING 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train),columns=X_train.columns)
X_train = X_train_scaled

***Feature selection***

In [ ]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier())

In [ ]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

5

In [ ]:
print(selected_feat)

Index(['time_s', 'x', 'y', 'z', 'magnitude'], dtype='object')


***RF CLASSIFIER with all features***

In [ ]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train) 

RandomForestClassifier(n_estimators=1000)

***Evaluation***

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
y_pred = clf.predict(X_test)

print("Accuracy: \t ", accuracy_score(y_test, y_pred))

print("\n \n-------------Classification Report-------------\n")
print(classification_report(y_test, y_pred))
print("\n \n-------------confusion matrix-------------\n")
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

Accuracy: 	  0.499178522291502

 
-------------Classification Report-------------

              precision    recall  f1-score   support

           0       0.51      0.22      0.31    276017
           1       0.50      0.79      0.61    270559

    accuracy                           0.50    546576
   macro avg       0.50      0.50      0.46    546576
weighted avg       0.50      0.50      0.46    546576


 
-------------confusion matrix-------------



array([[ 60270, 215747],
       [ 57990, 212569]])

In [ ]:
X_train.columns
selected_feat

Index(['time_s', 'x', 'y', 'z', 'magnitude'], dtype='object')

In [ ]:
y_test

101036    0
101037    0
101038    0
101039    0
101040    0
         ..
647607    0
647608    0
647609    0
647610    0
647611    0
Name: label, Length: 546576, dtype: int8

In [ ]:
y_pred

array([0, 0, 1, ..., 1, 1, 1], dtype=int8)

## ***SVM CLASSIFIER***

In [ ]:
#Import svm model
from sklearn import svm
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel
clf.fit(X_train, y_train) 

SVC(kernel='linear')

# ***EVALUATION***

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)

print("Accuracy: \t ", accuracy_score(y_test, y_pred))

print("\n \n-------------Classification Report-------------\n")
print(classification_report(y_test, y_pred))
print("\n \n-------------confusion matrix-------------\n")
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)